In [1]:
%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [2]:
from PIL import Image, ImageDraw

In [ ]:
import pkgutil

In [ ]:
import importlib
import inspect

In [ ]:
import waveshare_epd
from importlib import import_module
non_hd = []
for i in pkgutil.iter_modules(waveshare_epd.__path__):
    non_hd.append(i.name)


In [ ]:
for i in non_hd:
    
    if i.startswith('epd') and not i.startswith('epdconfig'):
        print(i)
        myepd = import_module(f'waveshare_epd.{i}')
        try:
            print('display: ',inspect.getfullargspec(myepd.EPD.display).args)
        except AttributeError:
            try:
                print('DISPLAY: ', inspect.getfullargspec(myepd.EPD.Display).args)
            except AttributeError:
                print('display: unknown')
        try:
#             print('clear: ', inspect.getfullargspec(myepd.EPD.clear).args)
#         except AttributeError:
            print('CLEAR: ', inspect.getfullargspec(myepd.EPD.Clear).args)
        except AttributeError:
            print('clear: unknown')
            
        print('\n')

In [3]:
def strict_enforce(*types):
    """strictly enforce type compliance within classes
    
    Usage:
    @strict_enforce(type1, type2, (type3, type4))
    def foo(val1, val2, val4):
        ...
    """
    def decorator(f):
        def new_f(self, *args, **kwds):
            #we need to convert args into something mutable   
            newargs = []        
            for (a, t) in zip(args, types):
                if not isinstance(a, t):
                    raise TypeError(f'"{a}" is not type {t}')
#                 newargs.append( t(a)) #feel free to have more elaborated convertion
            return f(self, *args, **kwds)
        return new_f
    return decorator

In [4]:
class ScreenError(Exception):
    pass

In [69]:
class Screen():
    def __init__(self, epd='None', rotation=0, mode='1', vcom=0.0):
        self.vcom = vcom
        self.mode = mode
        self.image = None
        self.hd = False
        self.resolution = []
        self.HD = False
        self.epd = epd
        self.rotation = rotation
        
        
    @property
    def vcom(self):
        return self._vcom

    @vcom.setter
    @strict_enforce(float)
    def vcom(self, vcom):
        if vcom==0:
            self._vcom = None
        elif vcom > 0:
            raise ValueError(f'vcom must be a negative float value: {vcom}')
        else:
            self._vcom = vcom

    @property
    def epd(self):
        return self._epd

    @epd.setter
    @strict_enforce(str)
    def epd(self, epd):
        myepd = None
        if epd=='HD':
            if not self.vcom:
                raise ScreenError('no vcom value is set (see the cable on your display for a vcom value)')
            self.HD = True
            myepd = self._epd_hd(epd)
            
        elif epd == 'None':
            myepd = None
        else:
            myepd = self._epd_non_hd(epd)

        if myepd:
            # set the resolution 
            self._epd = myepd['epd']
            resolution = myepd['resolution']
            resolution.sort(reverse=True)
            self.resolution = resolution
            
            # set a blank image as default
            self.image = Image.new('L', self.resolution, 255)
            
    def initEPD(self):
        '''init the EPD for writing'''
        if not self.epd:
            raise UnboundLocalError('no epd object has been assigned')
            
        if self.HD:
            self._epd.epd.run()
        else:
            try:
                self.epd.init()
            except Exception as e:
                logging.error(f'failed to init epd: {e}')
            else:
                logging.info(f'{self.epd} initialized')
            
        return True            

    def clearEPD(self):
        if self.HD:
            self._epd.epd.run()
            self._epd.clear()
        else:
            try:
                self.initEPD()
                self.epd.Clear(**self.clear_args)
            except Exception as e:
                logging.error(f'failed to clear epd: {e}')
                return False
        return True
        
    
    def blank_image(self):
        '''generate PIL image that is entirely blank'''
        return Image.new(self.mode, self.resolution, 255)
            
            
    def _epd_hd(self, epd):
        from IT8951.display import AutoEPDDisplay
        from IT8951 import constants as constants_HD
        myepd = AutoEPDDisplay(vcom=self.vcom)
        resolution = list(myepd.display_dims)
        clear_args = {}
        one_bit_display = False
        
        return {'epd': myepd, 
                'resolution': resolution, 
                'clear_args': clear_args, 
                'one_bit_display': one_bit_display}
            
    def _epd_non_hd(self, epd):
        import waveshare_epd
        import pkgutil
        import inspect
        from importlib import import_module
        non_hd = []
        for i in pkgutil.iter_modules(waveshare_epd.__path__):
            non_hd.append(i.name)

        if epd in non_hd:
            myepd = import_module(f'waveshare_epd.{epd}')
            resolution = [myepd.EPD_HEIGHT,myepd.EPD_WIDTH]
            
            # set kwargs for screens that expect color or mode arguments to the clear function
            try:
                clear_args_spec = inspect.getfullargspec(myepd.EPD.Clear)
            except AttributeError:
                raise ScreenError(f'"{epd}" has an unsupported `EPD.Clear()` function')
            clear_args = {}
            if 'color' in clear_args_spec:
                clear_args['color'] = 0xFF
                
            try:
                display_args_spec = inspect.getfullargspec(myepd.EPD.display)
            except AttributeError:
                raise ScreenError(f'"{epd}" has an unsupported `EPD.display()` function')
            if len(display_args_spec) > 2:
                one_bit_display = True
            else:
                one_bit_display = False
        else:
            raise ScreenError(f'invalid waveshare module: {epd}')
            
        return {'epd': myepd.EPD(), 
                'resolution': resolution, 
                'clear_args': clear_args,
                'one_bit_display': one_bit_display}

In [71]:
s = Screen(epd='HD', vcom=-1.93)

In [72]:
s.clearEPD()

True

In [60]:
q = s.epd

In [63]:
q.epd.sleep()


In [66]:
q.epd.run()

In [67]:
q.clear()

In [ ]:
s = Screen(vcom=-1.93, epd='HD')


In [ ]:
s.resolution

In [ ]:
from IT8951.display import AutoEPDDisplay as AEDD

In [ ]:
help (AEDD)

In [ ]:
q.clear()

In [ ]:
q = [100, 2]

In [ ]:
q.sort(reverse=True)

In [ ]:
q